# IMPORTS

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, train_test_split
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

## LEITURA DOS DADOS

In [15]:
df = pd.read_csv('/home/cid34senhas/Desktop/DAATP/RawData/train_radiomics_hipocamp.csv') 
df_test = pd.read_csv('/home/cid34senhas/Desktop/DAATP/RawData/test_radiomics_hipocamp.csv')


print(df['Transition'])

0        CN-CN
1        CN-CN
2        AD-AD
3       CN-MCI
4        CN-CN
        ...   
300      CN-CN
301      CN-CN
302     MCI-AD
303    MCI-MCI
304      CN-CN
Name: Transition, Length: 305, dtype: object


# Tratamento

### Single Value Colums 

In [16]:
df = df.loc[:, df.nunique() > 1]
df_test = df_test.loc[:, df_test.nunique() > 1]

#  Analise das colunas que tem menos de 50 valores unicos 
n = df.nunique()
#for col, e in n.items():
#    if e < 50:  
#        print(f"Coluna: {col}, Valores Unicos : {e}")

### AGE BINING  ( secalhar isto era no 2 )

In [17]:

age_bins = [0, 65, 75, 85, 100]
# BINS_SIZER = ['<65', '65-74', '75-84', '85+']
age_labels = [60, 70, 80, 90] # VALOR MEDIO DO BIN 
df['Age'] = pd.cut(df['Age'], bins=age_bins, labels=age_labels).astype(int)
df_test['Age'] = pd.cut(df_test['Age'], bins=age_bins, labels=age_labels).astype(int)

#print(df['Age'])



### Colunas Categoricas 

In [18]:
# Analisar a contagem de valores únicos para cada coluna categórica

# Identificar as colunas categóricas
categorical_columns = df.select_dtypes(include=['object', 'category']).columns

#for col in categorical_columns:
#    print(f"\nColuna: {col}")

#colunas_catagoricas_a_remover = ['ID', 'Image', 'Mask', 'diagnostics_Image-original_Hash', 'diagnostics_Mask-original_Hash'] 

colunas_catagoricas_a_remover = ['ID', 'Image', 'Mask', 'diagnostics_Image-original_Hash', 'diagnostics_Mask-original_Hash', 'diagnostics_Mask-original_BoundingBox', 'diagnostics_Mask-original_CenterOfMassIndex', 'diagnostics_Mask-original_CenterOfMass'] 

# ** Bounding Box
#
# ** as colunas do 'diagnostics_Mask-original_BoundingBox', 'diagnostics_Mask-original_CenterOfMassIndex', 'diagnostics_Mask-original_CenterOfMass'
# ** Deveriam ser retiradas, mas o bounding box pode ser importante para a zona de maior ativação do Alzimeir 
# **  ja a de centro de maxima devem ser muito correlacionados, por isso devem ser retirados mais para a frente 
# *TODO acabei por retirar para correr melhor os modelos, mas analisar se se deve retirar ou nao 

df.drop(columns=colunas_catagoricas_a_remover,axis= 1 , inplace= True)
df_test.drop(columns=colunas_catagoricas_a_remover,axis= 1 , inplace= True)

### Eliminar Transition CN-MCI 

Como este tipo de precisao não vai ser realiza, decidimos retirar todas as linhas com ela relacinadas para nao confundir o nosso modelo

In [19]:
# df = df[df['Transition'] != 'CN-MCI']

### Label Encoding

In [20]:
label_mapping = {
    'CN-CN': 0,
    'AD-AD': 1,
    'MCI-AD': 2,
    'MCI-MCI': 3,
    'CN-MCI' : 4
}


# Apply the mapping to the target column
#df['Transition'] = df['Transition'].map(label_mapping)

## Models

In [21]:

#experiment = setup(df, target='Transition')

In [22]:
#best = compare_models()

Com o tratamento atual, o pycaret diz que os melhores modelos são :
- Random Forest Classifier
- Extra Trees Classifier
- Gradient Boosting Classifier 
- K Neighbors Classifier
- Extreme Gradient Boosting 


### Aplicação dos modelos (default) ao dataset tratado

In [23]:
print(df['Transition'].dtype)
print(df['Transition'])

df['Transition'] = df['Transition'].astype(str)

df[['Origem', 'Destino']] = df['Transition'].str.split('-', expand=True)




object
0        CN-CN
1        CN-CN
2        AD-AD
3       CN-MCI
4        CN-CN
        ...   
300      CN-CN
301      CN-CN
302     MCI-AD
303    MCI-MCI
304      CN-CN
Name: Transition, Length: 305, dtype: object


In [24]:
def plot_confusion_matrix_with_labels(confusion_matrix, label_mapping):
    # Criar um mapeamento inverso
    reverse_label_mapping = {v: k for k, v in label_mapping.items()}
    
    # Aplicar o mapeamento inverso na matriz de confusão
    cm_with_labels = np.zeros_like(confusion_matrix, dtype=object)
    for i in range(confusion_matrix.shape[0]):
        for j in range(confusion_matrix.shape[1]):
            cm_with_labels[i, j] = f"{reverse_label_mapping[i]} (Pred: {reverse_label_mapping[j]})"

    # Criar um DataFrame para facilitar a visualização
    df_cm = pd.DataFrame(confusion_matrix, index=reverse_label_mapping.values(), columns=reverse_label_mapping.values())
    
    # Plotar a matriz de confusão
    plt.figure(figsize=(10, 7))
    sns.heatmap(df_cm, annot=True, fmt="d", cmap="Blues")
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix with Labels')
    plt.show()

In [25]:
from sklearn.discriminant_analysis import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix


# Separar as features e os targets
X = df.drop(columns=['Origem', 'Destino', 'Transition'])
y_origem = df['Origem']
y_destino = df['Destino']
y_transtion = df['Transition']
# Dividir o conjunto de dados em treino e teste para validação
X_train, X_test, y_train, y_test = train_test_split(X, y_origem, test_size=0.40, random_state=34, stratify=y_origem)

# Criar o modelo Random Forest
rf_origem_model = RandomForestClassifier(n_estimators=800, random_state=34)


# Avaliar desempenho com cross-validation no treino
rf_score_origem = cross_val_score(rf_origem_model, X_train, y_train, cv=5, scoring='f1_macro')
print("F1 Macro (Cross-Validation):", rf_score_origem.mean())

# Treinar o modelo no conjunto de treino e prever no conjunto de teste
rf_origem_model.fit(X_train, y_train)
rf_origem_pred = rf_origem_model.predict(X_test)

# Avaliar o desempenho no conjunto de teste
print("Relatório de Classificação (Teste):")
print(classification_report(y_test, rf_origem_pred))

# Confusion Matrix
#cm = confusion_matrix(y_test, rf_origem_pred)
#plot_confusion_matrix_with_labels(cm, label_mapping)

# Prever para todo o dataset usando cross-validation (sem vazamento)
df['Origem_Prevista'] = cross_val_predict(rf_origem_model, X, y_origem, cv=5)

# Adicionar Origem_Prevista como feature para Destino
X_destino = X.copy()
X_destino['Origem_Prevista'] = df['Origem_Prevista']

label_origem_encoding = {
    'CN': 0,
    'AD': 1,
    'MCI': 2,

}


X_destino['Origem_Prevista'] = X_destino['Origem_Prevista'].map(label_origem_encoding)


# Dividir o conjunto de dados para prever o destino
X_train, X_test, y_train, y_test = train_test_split(X_destino, y_destino, test_size=0.40, random_state=34, stratify=y_destino)

# Criar e avaliar o modelo Random Forest para Destino
rf_destino_model = RandomForestClassifier(n_estimators=800, random_state=34)
rf_score_destino = cross_val_score(rf_destino_model, X_train, y_train, cv=5, scoring='f1_macro')
print("F1 Macro (Cross-Validation para Destino):", rf_score_destino.mean())

# Treinar o modelo para prever o destino
rf_destino_model.fit(X_train, y_train)

# Fazer predições no conjunto de teste
rf_destino_pred = rf_destino_model.predict(X_test)

# Avaliar o desempenho no conjunto de teste
print("Relatório de Classificação (Destino):")
print(classification_report(y_test, rf_destino_pred))

# Matriz de confusão para o destino
#cm_destino = confusion_matrix(y_test, rf_destino_pred)
#plot_confusion_matrix_with_labels(cm_destino, label_mapping)

# Prever o destino para todo o dataset usando cross-validation
df['Destino_Previsto'] = cross_val_predict(rf_destino_model, X_destino, y_destino, cv=5)

# Exibir as predições completas
print(df[['Origem', 'Origem_Prevista', 'Destino', 'Destino_Previsto']].head())



## Normalizar os dados usando StandardScaler
#scaler = StandardScaler()
#
## Ajustar o scaler apenas nos dados de treino e transformar os dados de treino e teste
#X_train_normalizado = scaler.fit_transform(X_train)
#X_test_normalizado = scaler.transform(X_test)

# Exibir formas e primeiras linhas dos dados normalizados


# Inicializar dicionário para armazenar as pontuações de cross-validation do modelo
model_cross_score = {}
z1_score = {}




F1 Macro (Cross-Validation): 0.5168790952644908
Relatório de Classificação (Teste):
              precision    recall  f1-score   support

          AD       0.56      0.38      0.45        24
          CN       0.62      0.69      0.65        42
         MCI       0.56      0.59      0.57        56

    accuracy                           0.58       122
   macro avg       0.58      0.55      0.56       122
weighted avg       0.58      0.58      0.58       122



ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/home/cid34senhas/miniconda3/envs/DAA123/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/cid34senhas/miniconda3/envs/DAA123/lib/python3.12/site-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/cid34senhas/miniconda3/envs/DAA123/lib/python3.12/site-packages/sklearn/ensemble/_forest.py", line 363, in fit
    X, y = self._validate_data(
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/cid34senhas/miniconda3/envs/DAA

In [39]:
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.model_selection import cross_val_predict, train_test_split
import pandas as pd

def avaliar_modelos(df, modelo_origem_class, modelo_destino_class, 
                    params_origem=None, params_destino=None, 
                    label_mapping_origem=None, label_mapping_destino=None):
    """
    Avalia modelos de machine learning para Origem e Destino, com construção dinâmica dos modelos.
    
    Parâmetros:
    - df: DataFrame contendo as colunas 'Origem', 'Destino', 'Transition' e as features.
    - modelo_origem_class: Classe do modelo a ser usado para prever Origem (ex.: RandomForestClassifier).
    - modelo_destino_class: Classe do modelo a ser usado para prever Destino (ex.: XGBClassifier).
    - params_origem: Dicionário de parâmetros para o modelo de Origem.
    - params_destino: Dicionário de parâmetros para o modelo de Destino.
    - label_mapping_origem: Dicionário para mapear Origem_Prevista (opcional).
    - label_mapping_destino: Dicionário para mapear Destino_Previsto (opcional).
    
    Retorna:
    - Um dicionário com:
        - F1-scores para Origem e Destino.
        - Matrizes de confusão para Origem e Destino.
    """
    # Inicializar parâmetros, se não fornecidos
    params_origem = params_origem or {}
    params_destino = params_destino or {}

    # Criar os modelos dinamicamente
    modelo_origem = modelo_origem_class(**params_origem)
    modelo_destino = modelo_destino_class(**params_destino)

    # Separar as features e os targets
    X = df.drop(columns=['Origem', 'Destino', 'Transition'])
    y_origem = df['Origem']
    y_destino = df['Destino']
    y_origem = y_origem.map(label_mapping_origem)
    print(y_origem)
    X_train, X_test, y_train, y_test = train_test_split(X, y_origem, test_size=0.40, random_state=34, stratify=y_origem)

    score_origem = cross_val_score(modelo_origem, X_train, y_train, cv=5, scoring='f1_macro')

    df['Origem_Prevista'] = score_origem

    modelo_origem.fit(X_train, y_train)
    origem_pred = modelo_origem.predict(X_test)

    # Avaliar o desempenho para Origem

    print("Relatório de Classificação (Teste):")
    print(classification_report(y_test, rf_origem_pred))

    # Adicionar 'Origem_Prevista' como feature para Destino
    X_destino = X.copy()
    if label_mapping_destino:
        df['Origem_Prevista'] = df['Origem_Prevista'].map(label_mapping_origem)
    X_destino['Origem_Prevista'] = df['Origem_Prevista']

    # Previsão para Destino usando cross-validation
    df['Destino_Previsto'] = cross_val_predict(modelo_destino, X_destino, y_destino, cv=5)

    # Avaliar o desempenho para Destino
    f1_destino = f1_score(y_destino, df['Destino_Previsto'], average='macro')
    cm_destino = confusion_matrix(y_destino, df['Destino_Previsto'])

    # Resultado consolidado
    resultado = {
        "F1 Origem": f1_origem,
        "F1 Destino": f1_destino,
        "Matriz de Confusão Origem": cm_origem,
        "Matriz de Confusão Destino": cm_destino,
    }

    return resultado




In [ ]:
resultados = avaliar_modelos(
    df,
    modelo_origem_class=RandomForestClassifier,
    modelo_destino_class=RandomForestClassifier,
    params_origem={'n_estimators': 800, 'random_state': 34},
    params_destino={'n_estimators': 800, 'random_state': 34},
    label_mapping_origem={'CN': 0, 'AD': 1, 'MCI': 2},
    label_mapping_destino={'CN': 0, 'AD': 1, 'MCI': 2}
)

print("F1 Score Origem:", resultados["F1 Origem"])
print("F1 Score Destino:", resultados["F1 Destino"])
print("Matriz de Confusão Origem:\n", resultados["Matriz de Confusão Origem"])
print("Matriz de Confusão Destino:\n", resultados["Matriz de Confusão Destino"])